In [1]:
import pandas as pd
import numpy as np

from datetime import date

### Importando dataset e-commerce

Para mas informacion del dataset, consulte en el siguiente [link](https://www.kaggle.com/datasets/carrie1/ecommerce-data) 

In [2]:
ecommerce = pd.read_csv("Data.csv")

In [3]:
ecommerce.shape

(541909, 8)

### cast de fechas y tratamiento de columnas Dias, mes, año

In [4]:
ecommerce.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [5]:
ecommerce["InvoiceDate"] = pd.to_datetime(ecommerce["InvoiceDate"])

In [6]:
ecommerce.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom


In [8]:
ecommerce["year"] = ecommerce["InvoiceDate"].dt.year
ecommerce["month"] = ecommerce["InvoiceDate"].dt.month
ecommerce["day"] = ecommerce["InvoiceDate"].dt.day

In [9]:
ecommerce.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1


#### Exploracion de paises

In [10]:
ecommerce["Country"].drop_duplicates()

0               United Kingdom
26                      France
197                  Australia
385                Netherlands
1109                   Germany
1236                    Norway
1404                      EIRE
5320               Switzerland
6421                     Spain
6608                    Poland
7134                  Portugal
7214                     Italy
7279                   Belgium
7986                 Lithuania
9783                     Japan
14938                  Iceland
20000          Channel Islands
20017                  Denmark
29732                   Cyprus
30078                   Sweden
31464                  Austria
31982                   Israel
34083                  Finland
38313                  Bahrain
69007                   Greece
69623                Hong Kong
70758                Singapore
72985                  Lebanon
89570     United Arab Emirates
100810            Saudi Arabia
103598          Czech Republic
119191                  Canada
152712  

### Ejercicios

**Ejercicio 1:** Por pais muestre cual fue el año en el que mejor ingreso se registro y su valor, adicional muestre cual fue el año con mas Invoices (Frecuencia) y otra columna con el total.

Nota: Note que el año con mejor ingreso no es lo mismo con el año con mas invoices, por ejemplo en 2019 Honduras en dos invoices se facturo 10,000. sin embargo, en 2020 se realizaron 20 invoices (pueda que en este año el ingreso de esas invoices hayan sumado 7,000 inferior al 2020) 

*recuerde que el ejemplo es inventado*

Ejemplo de como se deberia de ver el dataframe resultante:


In [11]:
pd.DataFrame({
    "country" : ["Honduras"]
    , "YearMejorIngreso" : [2019]
    , "ValorYearMejorIngreso" : [20000]
    , "YearMejorFrecuenciaInvoices": [2020]
    , "CantidadYearMejorFrecuenciaInvoices": [20]
})

,country,YearMejorIngreso,ValorYearMejorIngreso,YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,Honduras,2019,20000,2020,20


*Answer:*

In [12]:
ecommerce['SubPrice'] = ecommerce.apply(lambda r: r['Quantity'] * r['UnitPrice'], axis=1)

In [13]:
oneA = ecommerce.copy()

In [14]:
oneA = oneA.loc[:, ['InvoiceNo','InvoiceDate', 'CustomerID', 'Country', 'year', 'month', 'day', 'SubPrice']]

In [15]:
oneA = oneA.groupby(['InvoiceDate', 'CustomerID', 'Country', 'year', 'month', 'day'], as_index=False).agg({'SubPrice': ['sum']})

In [16]:
oneA.columns = oneA.columns.droplevel(1)

In [17]:
oneA = oneA.loc[:, ['Country', 'year', 'SubPrice']]

In [18]:
oneA = oneA.groupby(['Country', 'year'], as_index=False).agg({'SubPrice': ['sum']})

In [19]:
oneA.columns = oneA.columns.droplevel(1)

In [20]:
oneA = oneA.sort_values(['SubPrice'], ascending=False)

In [21]:
oneA['order'] = oneA.groupby(['Country'], as_index=False).cumcount()

In [22]:
oneA = oneA.loc[oneA['order'] == 0, ['Country','year', 'SubPrice']]

In [23]:
oneA.columns = ['Country', 'YearMejorIngreso', 'ValorYearMejorIngreso']

In [24]:
oneB = ecommerce.copy()

In [25]:
oneB = oneB.groupby(['Country', 'year'], as_index=False).agg({'InvoiceNo':['count']})

In [26]:
oneB.columns = oneB.columns.droplevel(1)

In [27]:
oneB.columns = ['Country', 'year', 'Frecuencia']

In [28]:
oneB = oneB.sort_values(['Frecuencia'], ascending=False)

In [29]:
oneB['cumCount'] = oneB.groupby(['Country'], as_index=False).cumcount()

In [30]:
oneB = oneB.loc[oneB['cumCount'] == 0, ['Country', 'year', 'Frecuencia']]

In [31]:
oneB.columns = ['Country', 'YearMejorFrecuenciaInvoices', 'CantidadYearMejorFrecuenciaInvoices']

In [32]:
mergerd = pd.merge(oneA, oneB, left_on='Country', right_on='Country')

In [33]:
mergerd = mergerd.loc[:, ['Country', 'YearMejorIngreso', 'ValorYearMejorIngreso', 'YearMejorFrecuenciaInvoices', 'CantidadYearMejorFrecuenciaInvoices']]

In [34]:
mergerd

,Country,YearMejorIngreso,ValorYearMejorIngreso,YearMejorFrecuenciaInvoices,CantidadYearMejorFrecuenciaInvoices
0,United Kingdom,2011,6284073.654,2011,455353
1,Netherlands,2011,275877.060,2011,2299
2,EIRE,2011,242459.650,2011,7793
3,Germany,2011,207135.370,2011,8930
4,France,2011,187137.480,2011,8118
5,Australia,2011,136072.170,2011,1224
6,Switzerland,2011,54434.480,2011,1953
7,Spain,2011,52930.850,2011,2458
8,Belgium,2011,39101.050,2011,1974
9,Sweden,2011,33949.610,2011,435


**Ejercicio 2:** Cual es el Top tres por año de los Invoices con mas facturado

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado):

In [35]:
pd.DataFrame({
    "Year" : [2019,2019,2019,2020,2020,2020]
    , "Top" : [1,2,3,1,2,3]
    , "InvoiceNo" : [536365, 536999, 693365, 24987, 56832, 254232]    
    , "TotalInvoice": [30000,28000,27900,29000,25000,24600]
})

,Year,Top,InvoiceNo,TotalInvoice
0,2019,1,536365,30000
1,2019,2,536999,28000
2,2019,3,693365,27900
3,2020,1,24987,29000
4,2020,2,56832,25000
5,2020,3,254232,24600


*Answer:*

In [36]:
two = ecommerce.copy()

In [37]:
two = two.loc[:, ['InvoiceNo', 'year', 'month', 'day', 'SubPrice']]

In [38]:
two = two.groupby(['InvoiceNo', 'year'], as_index=False).agg({'SubPrice': ['sum']})

In [39]:
two.columns = two.columns.droplevel(1)

In [40]:
two.columns = ['InvoiceNo', 'year', 'TotalInvoice']

In [41]:
two = two.sort_values(['year', 'TotalInvoice'], ascending=False)

In [42]:
two['top'] = two.groupby(['year'],as_index=False).cumcount()

In [43]:
two = two.loc[(two['top'] >= 0) & (two['top'] <= 2),:]

In [44]:
two = two.loc[:, ['year', 'top', 'InvoiceNo', 'TotalInvoice']]

In [45]:
two

,year,top,InvoiceNo,TotalInvoice
22025,2011,0,581483,168469.60
2303,2011,1,541431,77183.60
18776,2011,2,574941,52940.94
1645,2010,0,539750,18745.86
674,2010,1,537659,15885.49
652,2010,2,537632,13541.33


**Ejercicio 3:** Genere el siguiente Dataset para cada uno de los clientes:

In [46]:
pd.DataFrame({
    "CustomerID" : [111,111,111]
    , "YearMonth" : ["2019-1","2019-2","2019-3"]
    , "TotalValorInvoices" : [20000,22000,0]    
    , "TotalCantidadInvoices" : [3,2,0]    
    , "FacturaSiguienteMes": [1,0,1]
})

,CustomerID,YearMonth,TotalValorInvoices,TotalCantidadInvoices,FacturaSiguienteMes
0,111,2019-1,20000,3,1
1,111,2019-2,22000,2,0
2,111,2019-3,0,0,1


*Observaciones:*

* TotalValorInvoices: La suma del total facturado por todas sus facturas
* TotalCantidadInvoices: La cantidad de facturas unicas realizadas en ese mes
* FacturaSiguienteMes: Una variable de respuesta que nos dice si al siguiente mes por ese cliente se encontro por lo menos una factura.

Note que por la logica de este dataset el ultimo año-mes encontrado en todo el dataset se va a descartar del resultado final por que no se sabe el valor del siguiente mes.

*Answer:*

In [47]:
three = ecommerce.copy()

In [48]:
three['YearMonth'] = three['year'].astype(str) + '-' + three['month'].astype(str)

In [49]:
three = three.groupby(['CustomerID', 'YearMonth'], as_index=False).agg({'SubPrice': ['sum'], 'InvoiceNo': ['count']})

In [50]:
three.columns = three.columns.droplevel(1)

In [51]:
three.columns = ['CustomerID', 'YearMonth', 'TotalValorInvoice', 'TotalCantidadInvoice']

In [52]:
three ['prevCustomerID'] = three['CustomerID'].shift(-1)

In [53]:
three['FacturaSiguienteMes'] = three.apply(lambda r: 1 if ((r['CustomerID'] == r['prevCustomerID']) & (r['TotalValorInvoice'] > 0.00) | (r['TotalValorInvoice'] < 0.00)) else 0, axis=1)

In [54]:
three = three.loc[:, ['CustomerID', 'YearMonth', 'TotalValorInvoice', 'TotalCantidadInvoice', 'FacturaSiguienteMes']]

In [55]:
three

,CustomerID,YearMonth,TotalValorInvoice,TotalCantidadInvoice,FacturaSiguienteMes
0,12346.0,2011-1,0.00,2,0
1,12347.0,2010-12,711.79,31,1
2,12347.0,2011-1,475.39,29,1
3,12347.0,2011-10,1294.32,47,1
4,12347.0,2011-12,224.82,11,1
...,...,...,...,...,...
13670,18283.0,2011-6,307.53,111,1
13671,18283.0,2011-7,143.19,57,1
13672,18283.0,2011-9,134.90,47,0
13673,18287.0,2011-10,1072.00,41,1


**Ejercicio cuatro:** Cual es el top 3 de los productos que menos ingresos generan por Pais del ultimo año encontrado del dataset.

Ejemplo de como se deberia de ver el dataframe resultante (Recuerde que este es un ejemplo inventado): 

In [56]:
pd.DataFrame({
    "Country" : ["Canada","Canada","Canada","Spain","Spain","Spain"]
    , "Top" : [1,2,3,1,2,3]    
    , "StockCode": [546,6886,665,656,888,668]
    , "Description": ["Product X","Product K","Product J","Product C","Product D","Product U"]
    , "TotalOnInvoices": [2000,2100,4000,3000,3800,3900]
})

,Country,Top,StockCode,Description,TotalOnInvoices
0,Canada,1,546,Product X,2000
1,Canada,2,6886,Product K,2100
2,Canada,3,665,Product J,4000
3,Spain,1,656,Product C,3000
4,Spain,2,888,Product D,3800
5,Spain,3,668,Product U,3900


*Answer:*

In [57]:
four = ecommerce.copy()

In [58]:
four = four.loc[four['year'] == four['year'].max(),:]

In [59]:
four = four.groupby(['StockCode', 'Description', 'Country'], as_index=False).agg({'SubPrice': ['sum']})

In [60]:
four.columns = four.columns.droplevel(1)

In [61]:
four.columns = ['StockCode', 'Description', 'Country', 'TotalOnInvoices']

In [62]:
four = four.sort_values(['TotalOnInvoices', 'Country'])

**Excluyo resultados negativos en la columna TotalOnInvoices puesto que estos no representan ingresos sino, pérdidas.**

In [63]:
four = four.loc[four['TotalOnInvoices'] >= 0,:].copy()

In [64]:
four['top'] = four.groupby(['Country'],as_index=False).cumcount()

In [65]:
four = four.loc[(four['top'] >= 0) & (four['top'] <= 2 ), :]

In [66]:
four = four.sort_values(['Country','TotalOnInvoices','top'])

In [67]:
four = four.loc[:, ['Country', 'top', 'StockCode', 'Description', 'TotalOnInvoices']]

In [68]:
four

,Country,top,StockCode,Description,TotalOnInvoices
6098,Australia,0,22232,JIGSAW TOADSTOOLS 3 PIECE,0.00
9872,Australia,1,22738,RIBBON REEL SNOWY VILLAGE,0.00
11822,Australia,2,22990,COTTON APRON PANTRY DESIGN,0.00
2697,Austria,0,21402,RED EGG SPOON,2.88
2704,Austria,1,21403,BLUE EGG SPOON,2.88
...,...,...,...,...,...
22,United Kingdom,1,10133,damaged,0.00
112,United Kingdom,2,15058A,wet/rusty,0.00
17305,Unspecified,0,79063C,"RETRO PILL BOX KEY CHAIN,THE KING",0.19
7118,Unspecified,1,22396,MAGNETS PACK OF 4 RETRO PHOTO,0.39
